1. 预处理类别名称：汉化

In [ ]:
import translators as ts
from tqdm import tqdm

CID_MAP_FILE = "../data/256_ObjectCategories_map.csv"
TRANSLATOR = "sogou"
# TRANSLATOR = "alibaba"
ZH_CID_MAP_FILE = f"../data/256_ObjectCategories_map_{TRANSLATOR}.csv"

f_r = open(CID_MAP_FILE, "r")
f_w = open(ZH_CID_MAP_FILE, "w")

for i, line in tqdm(enumerate(f_r)):
    line = line.strip()
    if i == 0:
        f_w.write(line + ",class_ZH\n")
        continue
    class_name = line.strip().split(",")[1]
    class_name_ZH = ts.translate_text(class_name, 
                                      translator=TRANSLATOR, 
                                      from_language="en", 
                                      to_language="zh")
    # print(line + "," + class_name_ZH + "\n")
    f_w.write(line + "," + class_name_ZH + "\n")
    
f_r.close()
f_w.close()

'apertium',
  'argos',
  'baidu',
  'cloudYi',
  'deepl',
  'elia',
  'iflyrec',
  'judic',
  'languageWire',
  'lingvanex',
  'mglip',
  'modernMt',
  'myMemory',
  'papago',
  'sogou',
  'translateCom',
  'utibet'

In [ ]:
import translators as ts
import pandas as pd

CID_MAP_FILE = "../data/256_ObjectCategories_map.csv"
ZH_CID_MAP_FILE = f"../data/256_ObjectCategories_map_ZH.csv"

tranlators = ["translateCom"]
cid_map_df = pd.read_csv(CID_MAP_FILE, index_col="cid")

for translator in tranlators:
    cid_map_df[translator] = cid_map_df["class"].apply(lambda x: ts.translate_text(x, translator=translator, from_language="en", to_language="zh"))

cid_map_df.to_csv(ZH_CID_MAP_FILE)

2. 预处理特征

In [ ]:
import numpy as np

IMAGE_DATA_FILE = "../data/caltech-256_features.npz"

image_data = np.load(IMAGE_DATA_FILE)
print(list(image_data.keys()))


In [ ]:
X_ViT = image_data.get("vit_features")
print(f"The shape of X_ViT is {X_ViT.shape}")

X_CLIP = image_data.get("clip_features")
print(f"The shape of X_CLIP is {X_CLIP.shape}")

y = image_data.get("labels")
print(f"The shape of y is {y.shape}")


In [ ]:
np.linalg.norm(X, 2, axis=1)

In [ ]:
np.linalg.norm(X_ViT, 2, axis=1)


In [ ]:
np.linalg.norm(X_CLIP, 2, axis=1)
